In [0]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
%matplotlib inline


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
data = pd.read_csv('/content/drive/My Drive/assignment 2/Data/sales_train.csv')
test = pd.read_csv('/content/drive/My Drive/assignment 2/Data/test.csv')
submission = pd.read_csv('/content/drive/My Drive/assignment 2/Data/sample_submission.csv')
items = pd.read_csv('/content/drive/My Drive/assignment 2/Data/items.csv')
item_cats = pd.read_csv('/content/drive/My Drive/assignment 2/Data/item_categories.csv')
shops = pd.read_csv('/content/drive/My Drive/assignment 2/Data/shops.csv')

In [6]:
print ('Data Set') 
print(data.shape)

print ('Test Set') 
print(test.shape)


Data Set
(2935849, 6)
Test Set
(214200, 3)


In [5]:
data['date'] = pd.to_datetime(data['date'], format='%d.%m.%Y')
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year

data = data.drop(['date', 'item_price'], axis=1)
data = data.groupby([c for c in data.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
data = data.rename(columns={'item_cnt_day':'item_cnt_month'})

data.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month
0,0,0,32,1,2013,6.0
1,0,0,33,1,2013,3.0
2,0,0,35,1,2013,1.0
3,0,0,43,1,2013,1.0
4,0,0,51,1,2013,2.0


In [6]:
shop_item_mean = data[['shop_id', 'item_id', 'item_cnt_month']].groupby(['shop_id', 'item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_mean = shop_item_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
shop_prev_month = data[data['date_block_num'] == 33][['shop_id', 'item_id', 'item_cnt_month']]
shop_prev_month = shop_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_mean'})
data = pd.merge(data, shop_prev_month, how='left', on=['shop_id', 'item_id']).fillna(0.)
data = pd.merge(data, items, how='left', on='item_id')
data = pd.merge(data, item_cats, how='left', on='item_category_id')
data = pd.merge(data, shops, how='left', on='shop_id')
data.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_prev_mean,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,1,2013,6.0,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,33,1,2013,3.0,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
2,0,0,35,1,2013,1.0,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
3,0,0,43,1,2013,1.0,0.0,100 МИЛЛИОНОВ ЕВРО,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
4,0,0,51,1,2013,2.0,0.0,100 лучших произведений классики (mp3-CD) (Dig...,57,Музыка - MP3,"!Якутск Орджоникидзе, 56 фран"


In [7]:
test['month'] = 11
test['year'] = 2015
test['date_block_num']=34
test = pd.merge(test, shop_item_mean, how='left', on=['shop_id', 'item_id']).fillna(0.)
test = pd.merge(test, shop_prev_month, how='left', on=['shop_id', 'item_id']).fillna(0.)
test = pd.merge(test, items, how='left', on='item_id')
test = pd.merge(test, item_cats, how='left', on='item_category_id')
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_month_mean,item_cnt_prev_mean,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.444444,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0.0
2,2,5,5233,11,2015,34,2.000000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0.0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0.0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0.0


In [0]:
from sklearn import preprocessing
for c in ['shop_name', 'item_name', 'item_category_name']:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(data[c].unique())+list(test[c].unique()))
    data[c] = lbl.transform(data[c].astype(str))
    test[c] = lbl.transform(test[c].astype(str))

In [0]:
col = [c for c in data.columns if c not in ['item_cnt_month']]
x1 = data[data['date_block_num']<33]
y1 = x1['item_cnt_month']
x1=x1[col]
x2 = data[data['date_block_num'] == 33]
y2 = x2['item_cnt_month'].to_numpy()
x2=x2[col]

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(x1,y1) 
pred = clf.predict(x2)

In [13]:
from sklearn.metrics import accuracy_score
accuracy_score(y2, pred)

0.525736576702293

In [9]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import *
from keras.regularizers import l2
from keras.optimizers import *
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
def create_data(cat_features):
    lenUniquFeatures = {}
#     little_data = data.sample(frac = 0.4)
    X_test = test.loc[:,cat_features].copy()
    processed_data = data.loc[:,cat_features].copy()
    for cat_feature in cat_features:
        feature_inx ={p:i for (i,p) in enumerate(data[cat_feature].unique())}
        lenUniquFeatures[cat_feature] = len(feature_inx)
        processed_data[cat_feature] = [feature_inx[x] for x in data[cat_feature]]
        feature_inx_test ={p:i for (i,p) in enumerate(X_test[cat_feature].unique())}
        lenUniquFeatures[cat_feature] = len(feature_inx_test)
        X_test[cat_feature] = [feature_inx_test[x] for x in X_test[cat_feature]]
    target = data['item_cnt_month'].values
    return processed_data, target, X_test, lenUniquFeatures

In [0]:
cat_features = ['date_block_num', 'item_id', 'shop_id', 'item_category_id','month', 'year']#,'month_length'
processed_data, target, X_test, lenUniquFeatures = create_data(cat_features)

In [12]:
def build_categorical_inputs(features):
    initial_inputs = {}
    cat_input_layers={}
    for feature in features:         
        embedding_size = int(min(np.sqrt(lenUniquFeatures[feature]), 50))
        initial_inputs[feature] = Input(shape=(1,),dtype='int64')
        embedding_layer = Embedding(lenUniquFeatures[feature], 
                                    embedding_size,
                                    embeddings_regularizer=l2(0.001),
                                    input_length=1)(initial_inputs[feature])
        cat_input_layers[feature] = Reshape(target_shape=(embedding_size,))(embedding_layer)
    return initial_inputs, cat_input_layers
initial_inputs, input_layers = build_categorical_inputs(cat_features)

In [19]:
x = Concatenate(axis=-1)([layer for layer in input_layers.values()])
x = Dropout(0.6)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(64,activation='relu')(x)
x = Dense(1)(x)
inputs=[input for input in initial_inputs.values()]
print (inputs)
model = Model(inputs,outputs=x)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])#optimizer='adam',loss='binary_crossentropy',
              
model.summary()


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[<tf.Tensor 'input_1:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'input_2:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'input_3:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'input_4:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'input_5:0' shape=(?, 1) dtype=int64>, <tf.Tensor 'input_6:0' shape=(?, 1) dtype=int64>]

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
______________________________________________________________________

In [20]:
from sklearn.model_selection import KFold,StratifiedKFold
from keras.callbacks import *

checkpoint_path = os.getcwd()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = ModelCheckpoint(filepath='best_model_weights_run1.h5',save_weights_only=True,verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
tensor_board = TensorBoard(log_dir= checkpoint_path)
history = History()
features = processed_data.columns
# kf = StratifiedKFold(n_splits=5,random_state=42)
kf = KFold(n_splits=5,random_state=42)
for tr_ind,val_ind in kf.split(processed_data,target):
    X_train,X_val,y_train,y_val = processed_data.iloc[tr_ind],processed_data.iloc[val_ind],target[tr_ind],target[val_ind]
    X_trainA = []
    for cat in cat_features:
        X_trainA.append(np.array(X_train[cat]).reshape(-1,1))
    X_valA = []
    for cat in cat_features:
        X_valA.append(np.array(X_val[cat]).reshape(-1,1))
    history = model.fit(X_trainA,y_train,epochs=3,validation_data=[X_valA,y_val],callbacks=[checkpoint, early_stop],verbose=1) #set_callbacks())
    break

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning





Train on 1287299 samples, validate on 321825 samples
Epoch 1/3





1287299/1287299 [==============================] - 262s 204us/step - loss: 70.4899 - acc: 0.2427 - val_loss: 20.5897 - val_acc: 0.1416

Epoch 00001: saving model to best_model_weights_run1.h5
Epoch 2/3
1287299/1287299 [==============================] - 253s 196us/step - loss: 68.0929 - acc: 0.2429 - val_loss: 27.2473 - val_acc: 0.0593

Epoch 00002: saving model to best_model_weights_run1.h5
Epoch 3/3
1287299/1287299 [==============================] - 255s 198us/step - loss: 67.3037 - acc: 0.2446 - val_loss: 24.3838 - val_acc: 0.0926

Epoch 00003: saving model to best_model_weights_run1.h5


In [21]:
type(history.history)
hist= pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,val_loss,val_acc,loss,acc,epoch
0,20.589691,0.141624,70.489942,0.242676,0
1,27.247338,0.059330,68.092949,0.242890,1
2,24.383759,0.092594,67.303696,0.244637,2


In [0]:
cat_features = ['date_block_num', 'item_id', 'shop_id', 'item_category_id','month', 'year','shop_name', 'item_name', 'item_category_name']#,'month_length'
processed_data, target, X_test, lenUniquFeatures = create_data(cat_features)
moreFeatures = ['shop_name', 'item_name', 'item_category_name']
for more in moreFeatures:
    initial_inputs[more] = Input(shape=(1,))
    input_layers[more] = initial_inputs[more]

In [23]:
x = Concatenate(axis=-1)([layer for layer in input_layers.values()])
x = Dropout(0.6)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.6)(x)
x = Dense(64,activation='relu')(x)
x = Dense(1)(x)
inputs=[input for input in initial_inputs.values()]
model = Model(inputs,outputs=x)
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1)            0                                            
____________________________________________________________________________________________

In [24]:
from sklearn.model_selection import KFold,StratifiedKFold
from keras.callbacks import *

checkpoint_path = os.getcwd()
checkpoint_dir = os.path.dirname(checkpoint_path)
checkpoint = ModelCheckpoint(filepath='best_model_weights_run1.h5',save_weights_only=True,verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=3)
tensor_board = TensorBoard(log_dir= checkpoint_path)
history = History()
features = processed_data.columns
kf = StratifiedKFold(n_splits=5,random_state=42)
kf = KFold(n_splits=5,random_state=42)
for tr_ind,val_ind in kf.split(processed_data,target):
    X_train,X_val,y_train,y_val = processed_data.iloc[tr_ind],processed_data.iloc[val_ind],target[tr_ind],target[val_ind]
    X_trainA = []
    for cat in cat_features:
        X_trainA.append(np.array(X_train[cat]).reshape(-1,1))
    X_valA = []
    for cat in cat_features:
        X_valA.append(np.array(X_val[cat]).reshape(-1,1))
    history = model.fit(X_trainA,y_train,epochs=3,validation_data=[X_valA,y_val],callbacks=[checkpoint, early_stop],verbose=1) #set_callbacks())
    break

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Train on 1287299 samples, validate on 321825 samples
Epoch 1/3
1287299/1287299 [==============================] - 261s 203us/step - loss: 2752.0870 - acc: 0.1928 - val_loss: 23.0062 - val_acc: 0.1568

Epoch 00001: saving model to best_model_weights_run1.h5
Epoch 2/3
1287299/1287299 [==============================] - 258s 201us/step - loss: 85.0670 - acc: 0.1666 - val_loss: 23.2408 - val_acc: 0.0616

Epoch 00002: saving model to best_model_weights_run1.h5
Epoch 3/3
1287299/1287299 [==============================] - 257s 200us/step - loss: 89.7989 - acc: 0.1752 - val_loss: 22.9531 - val_acc: 0.1437

Epoch 00003: saving model to best_model_weights_run1.h5


In [25]:
type(history.history)
hist= pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist

,val_loss,val_acc,loss,acc,epoch
0,23.006240,0.156793,2752.086952,0.192802,0
1,23.240826,0.061633,85.067027,0.166633,1
2,22.953063,0.143740,89.798896,0.175163,2


In [0]:
processed_data1 = processed_data.year.astype(str).str.get_dummies()
processed_data1.columns = ['year' + str(x) for x in processed_data1.columns]
processed_data = processed_data.drop(['year'], axis=1)
processed_data = pd.concat([processed_data,processed_data1], axis=1)


In [0]:
msk = np.random.rand(len(processed_data)) < 0.8
X_train = processed_data[msk]
y_train = target[msk]
X_test = processed_data[~msk]
y_test = target[~msk]

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0).fit(X_train,y_train) 
pred = clf.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

0.5430822056150273